In [25]:
import pandas as pd
import numpy as np
import re
import ast

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
# nltk.download('stopwords',quiet=True)
# nltk.download('wordnet', quiet=True)
# nltk.download('punkt',quiet=True)

from thefuzz import fuzz
from thefuzz import process

from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

from scipy.sparse import csr_matrix

In [2]:
def data_pre_pre_processing(df:pd.DataFrame):
    df= (df[["wonum","description","ldtext","mats_assigned","wopriority","actstart"]]
         .drop_duplicates()
         .reset_index(drop=True)
    )
    df[["ldtext", "description"]] = df[["ldtext", "description"]].astype(str)
    df["actstart"] = pd.to_datetime(df["actstart"])

    un_wonums=(df[["wonum"]].drop_duplicates(keep=False))
    df=un_wonums.merge(df,on='wonum').reset_index(drop=True)

    return df

In [3]:
def text_pre_processing(text):

    # Remove numbers and punctuation
    clean_text = "".join([i for i in text if i.isalpha() or i.isspace()])
    # Remove exceess whitespace
    clean_text = re.sub(r'\s+', ' ', clean_text)
    # Transform to lower case
    clean_text = clean_text.lower()

    tokens = nltk.word_tokenize(clean_text)
    #Removestopwords and character-like words
    clean_tokens = [w for w in tokens if (not w in stopwords.words("english")) and (len(w) > 2)]

    # Lemmatizatize the words(not stemming as we will use doc2vec later on which captures the meaning of words, therefore stemming is not applicable in this case)
    wordnet_lemmatizer = WordNetLemmatizer()
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in clean_tokens]

    return lemm_text

data = pd.DataFrame({"text": ['u can !write up to ""3^*(%) 20GB to th^e current directory (/kaggle/working/) that gets preserved as output when you create a version using Save & Run All ']*int(10e1)})
data['clean_text'] = data['text'].apply(lambda x: text_pre_processing(x))
print(f'Original text:    {data["text"][0]}')
print(f'Transformed text:    {data["clean_text"][0]}')

Original text:    u can !write up to ""3^*(%) 20GB to th^e current directory (/kaggle/working/) that gets preserved as output when you create a version using Save & Run All 
Transformed text:    ['write', 'current', 'directory', 'kaggleworking', 'get', 'preserved', 'output', 'create', 'version', 'using', 'save', 'run']


In [4]:
def merge_text_cols(df: pd.DataFrame, col1:str , col2:str):
    
    try:
        for index, row in df.iterrows():
            lst1 = ast.literal_eval(row[col1])
            lst2 = ast.literal_eval(row[col2])
            for item in lst2:
                lst1.append(item)
            df.loc[index,f"{col1}_{col2}"] = str(lst1)
    except:
        for index, row in df.iterrows():
            lst1 = row[col1]
            lst2 = row[col2]
            for item in lst2:
                lst1.append(item)
    
        
    return df

In [5]:
def gather_relevant_mats(columns, df):

    def a2v_extraction(text):

        if text != 'nan':
            # Remove exceess whitespace
            clean_text = re.sub(r'\s+', ' ', text)
            pattern = r'[!"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~]'
            # Replace matched punctuation with a space
            clean_text = re.sub(pattern, ' ', clean_text)
            tokens = nltk.word_tokenize(clean_text)

            a2v_nums = [(w)
                        for w in tokens if w.startswith('A2V') and len(w) > 5]
        else:
            return ''

        return ",".join(list(set(a2v_nums)))

    mats_cols = []
    for col in columns:
        df[f'mats_from_{col}'] = df[col].apply(a2v_extraction)
        mats_cols.append(f'mats_from_{col}')

    df['all_relevant_mats'] = (df[mats_cols].astype(str).apply(','.join, axis=1)
                                            .apply(lambda x: list(set(x.split(','))))

                               )

    df = df.drop(columns=mats_cols)

    return df


In [6]:
def data_sanitising(df: pd.DataFrame):

    df["actstart"] = pd.to_datetime(df["actstart"])
    df["clean_ldtext_size"] = df["clean_ldtext"].apply(lambda x: len(x))
    df["clean_description_size"] = df["clean_description"].apply(lambda x: len(x))

    # Fix data quality
    df['clean_description'] = df['clean_description'].fillna('[]')
    df['clean_ldtext'] = df['clean_ldtext'].fillna('[]')
    df['ldtext'] = df['ldtext'].astype(str)
    df['description'] = df['description'].astype(str)
    df['clean_ldtext'] = df['clean_ldtext'].astype(str)
    df['clean_description'] = df['clean_description'].astype(str)
    
    df = df.replace({"['nan']": '[]', '[nan]': '[]'})
    df = df.drop(df[df["ldtext"].astype(str).apply(lambda x: x.startswith('[if gte mso 9]&'))].index,axis=0).reset_index(drop=True)

    #Merge clean_description and clean_ldtext
    df = merge_text_cols(df,"clean_description","clean_ldtext")
    cols = ['description', 'ldtext']
    df = gather_relevant_mats(cols, df) 
        
    return df
    

In [7]:
def exact_matches(df):
    
    df1 = df.groupby(["clean_description", "clean_ldtext"], dropna = False).count()
    df1 = (df1[df1["wonum"] > 1]["wonum"].reset_index(drop=False))
    df1 = df1.rename(columns={"wonum": "count"}).reset_index(drop=True)
    df1["group_id"] = df1.index 
    df = df.merge(df1,on= ["clean_description", "clean_ldtext"], how='left')
    
    group_map = (df.groupby("group_id", dropna=True)['wonum'].apply(lambda x: ",".join(x))
                 .reset_index()
                 .rename(columns={"wonum": "similar_clean_description_ldtext"})
                )
    df = df.merge(group_map, on=["group_id"], how='left')
    
    df["exact_match"] = np.where(df["group_id"].isna(),0,1)
    
    return df

In [21]:
df = pd.read_csv(r"brake_workorders.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6677 entries, 0 to 6676
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   wonum                 6677 non-null   object 
 1   imo_act_causing_part  4396 non-null   object 
 2   description           6677 non-null   object 
 3   workorderid           6677 non-null   int64  
 4   wopriority            5129 non-null   float64
 5   worktype              6677 non-null   object 
 6   actstart              6376 non-null   object 
 7   assetnum              5536 non-null   object 
 8   location              5706 non-null   object 
 9   siteid                6677 non-null   object 
 10  wonum.1               381 non-null    object 
 11  mats_assigned         381 non-null    object 
 12  ldkey                 4123 non-null   float64
 13  ldtext                4109 non-null   object 
dtypes: float64(2), int64(1), object(11)
memory usage: 730.4+ KB


In [22]:
df=data_pre_pre_processing(df)
df["clean_ldtext"] = df['ldtext'].apply(lambda x: text_pre_processing(x))
df["clean_description"] = df['description'].apply(lambda x: text_pre_processing(x))

df.head()

,wonum,description,ldtext,mats_assigned,wopriority,actstart,clean_ldtext,clean_description
0,1372443,Re-Instate Brake Smart Valve,nan,NaN,NaN,2018-10-20 19:11:57,[nan],"[reinstate, brake, smart, valve]"
1,TLW1660664,20/03 411 Brake Release Key Missing,401 cab lower cup holder damaged - ATTENDED no...,NaN,5.0,2020-03-21 12:35:02,"[cab, lower, cup, holder, damaged, attended, n...","[brake, release, key, missing]"
2,TLW2429000,Re-Instate Brake Smart Valve,nan,NaN,9.0,2024-01-11 15:10:43,[nan],"[reinstate, brake, smart, valve]"
3,1139804,Intercar Gangway Disconnected,nan,NaN,9.0,2017-08-01 12:48:49,[nan],"[intercar, gangway, disconnected]"
4,TLW1927638,411 - EP2002 Smart Valve Failing brake test to...,nan,NaN,9.0,2021-10-10 16:00:00,[nan],"[smart, valve, failing, brake, test, replaced,..."


In [24]:
df = data_sanitising(df)
df = exact_matches(df)
df.head(3)

,wonum,description,ldtext,mats_assigned,wopriority,actstart,clean_ldtext,clean_description,clean_ldtext_size,clean_description_size,clean_description_clean_ldtext,all_relevant_mats,count,group_id,similar_clean_description_ldtext,exact_match
0,1372443,Re-Instate Brake Smart Valve,nan,NaN,NaN,2018-10-20 19:11:57,[],"['reinstate', 'brake', 'smart', 'valve']",1,4,"['reinstate', 'brake', 'smart', 'valve']",[],1380.0,172.0,"1372443,TLW2429000,TLW2179413,TLW1824466,TLW24...",1
1,TLW1660664,20/03 411 Brake Release Key Missing,401 cab lower cup holder damaged - ATTENDED no...,NaN,5.0,2020-03-21 12:35:02,"['cab', 'lower', 'cup', 'holder', 'damaged', '...","['brake', 'release', 'key', 'missing']",10,4,"['brake', 'release', 'key', 'missing', 'cab', ...",[],NaN,NaN,NaN,0
2,TLW2429000,Re-Instate Brake Smart Valve,nan,NaN,9.0,2024-01-11 15:10:43,[],"['reinstate', 'brake', 'smart', 'valve']",1,4,"['reinstate', 'brake', 'smart', 'valve']",[],1380.0,172.0,"1372443,TLW2429000,TLW2179413,TLW1824466,TLW24...",1


In [11]:
df0 =df.copy()
df0['clean_description'] = df0['clean_description'].apply(lambda w: " ".join( ast.literal_eval(w)))

In [31]:
non_exact_ld_matches_df = df[(df["exact_match"] == 0)].reset_index(drop= True)
test_df = non_exact_ld_matches_df.copy()

non_exact_ld_matches_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3278 entries, 1 to 5629
Data columns (total 17 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   wonum                             3278 non-null   object        
 1   description                       3278 non-null   object        
 2   ldtext                            3278 non-null   object        
 3   mats_assigned                     279 non-null    object        
 4   wopriority                        3148 non-null   float64       
 5   actstart                          3086 non-null   datetime64[ns]
 6   clean_ldtext                      3278 non-null   object        
 7   clean_description                 3278 non-null   object        
 8   clean_ldtext_size                 3278 non-null   int64         
 9   clean_description_size            3278 non-null   int64         
 10  clean_description_clean_ldtext    3278 non-null   obj

In [32]:
def tf_idf_similarity_df(df:pd.DataFrame, col: str, vect_max_feats : int =500 ,n_splits: int = 10):
      
      if n_splits <= 0 or n_splits > len(df):
        raise ValueError(
            "n_splits should be a positive integer less than or equal to the length of the dataframe")

      q_sim_df = pd.DataFrame()

      corpus = [" ".join( (ast.literal_eval(text))) for text in df[col][0:int(len(df)/n_splits)]]
      vectorizer = TfidfVectorizer(ngram_range=(1,1), max_features = vect_max_feats)
      X = vectorizer.fit_transform(corpus).astype(np.float32)

      print(f'We have {len(corpus)} documents and {len(set(vectorizer.get_feature_names_out()))} unique words in our corpus.\n'
            f'Tf-idf matrix is a {X,X.dtype}') 

      
      sim_mat = cosine_similarity(X)#.astype(np.float32)
      q_sim_df = pd.DataFrame.sparse.from_spmatrix(csr_matrix(np.round(sim_mat.data, 2)))
      
      return q_sim_df


def get_cluster_info(des_mat: pd.DataFrame, df: pd.DataFrame, key_word: str, similarity: float):
    
      words = key_word.split(" ")
      # " ".join(text_pre_processing(key_word))
      matches = df[df["clean_description"].apply(lambda x: all(word in ast.literal_eval(x) for word in words))]

      if len(matches) != 0:
            print(f'Which description best matches what you are looking for?\n')
            print(matches['description'].head(50))
      else:
            print('No matches found in this dataset')
      
      key = input()

      return df.iloc[des_mat[des_mat[int(key)] > similarity].sort_values( int(key), ascending=False).index]


def get_similar_wonums(des_mat: pd.DataFrame, df: pd.DataFrame, index_key: int, similarity: float):

      matches = df.iloc[des_mat[des_mat[int(index_key)] > similarity].sort_values(int(index_key), ascending=False).index]
     
      if (len(matches) != 0) and (len(matches) != 1):
          print(f'Descriptions matched:\n')
          print(matches['description'].drop_duplicates().head(50))

      else:
          print('No matches found in this dataset')
          return None

      return ",".join(matches['wonum'])

In [33]:
n = len(non_exact_ld_matches_df) 
q_sim_df = tf_idf_similarity_df(df, col='clean_description', vect_max_feats=1000, n_splits=1)
similarity = 0.75
get_similar_wonums(q_sim_df, df, index_key=0, similarity=0.75)

We have 5630 documents and 1000 unique words in our corpus.
Tf-idf matrix is a (<5630x1000 sparse matrix of type '<class 'numpy.float32'>'
	with 27803 stored elements in Compressed Sparse Row format>, dtype('float32'))
Descriptions matched:

0       Re-Instate Brake Smart Valve
3622    Re-Instate Brake Smart Valve
3650    Re-Instate Brake Smart Valve
3649    Re-Instate Brake Smart Valve
3648    Re-Instate Brake Smart Valve
3645    Re-Instate Brake Smart Valve
3644    Re-Instate Brake Smart Valve
3631    Re-Instate Brake Smart Valve
3627    Re-Instate Brake Smart Valve
3626    Re-Instate Brake Smart Valve
3625    Re-Instate Brake Smart Valve
3619    Re-Instate Brake Smart Valve
3653    Re-Instate Brake Smart Valve
3618    Re-Instate Brake Smart Valve
3613    Re-Instate Brake Smart Valve
3610    Re-Instate Brake Smart Valve
3609    Re-Instate Brake Smart Valve
3608    Re-Instate Brake Smart Valve
3607    Re-Instate Brake Smart Valve
3604    Re-Instate Brake Smart Valve
3602    Re-Instate

'1372443,TLW1752580,TLW2018324,TLW2571872,TLW1618016,TLW2106313,TLW1719290,TLW2009400,TLW2626138,1448024,TLW1887486,TLW2092104,TLW1734981,1511249,TLW2412155,TLW2003339,TLW2632139,TLW1778755,TLW1520688,TLW2222383,TLW1632600,TLW2229808,TLW1676264,TLW2405347,1339216,TLW2239472,TLW2368265,TLW2145620,TLW1783025,TLW1645665,TLW2236372,TLW1791963,TLW1674462,TLW2531633,TLW1713426,TLW2406804,TLW1628611,TLW2312927,TLW2069984,TLW1804552,TLW1757125,1452042,TLW2285244,TLW2645010,TLW2155216,TLW2590817,TLW1679160,TLW2339111,TLW2328756,1458196,TLW2353186,TLW1745864,TLW2432063,TLW1916373,TLW1799103,TLW1537974,TLW2375176,TLW2313190,TLW2129271,TLW1791450,TLW2316285,TLW1565536,TLW1792946,TLW2251999,TLW1704235,TLW2534011,TLW1860877,TLW1850925,TLW2524110,TLW1960006,TLW2590749,TLW2008320,TLW2555131,TLW2559091,TLW1997923,TLW1646772,TLW1711487,TLW1853656,TLW2479031,1311491,TLW2453613,TLW2391764,TLW2263974,TLW2136047,TLW2375214,TLW1825350,TLW2403164,TLW2257879,TLW2420996,TLW1676715,TLW2320111,TLW1601671,TLW20957

*Below we'll output an example cluster of workorders matched based on their description*

In [34]:
get_cluster_info(q_sim_df,df,'brake',similarity=0.8)

Which description best matches what you are looking for?

0                          Re-Instate Brake Smart Valve
1                   20/03 411 Brake Release Key Missing
2                          Re-Instate Brake Smart Valve
4     411 - EP2002 Smart Valve Failing brake test to...
5     Bogie 2 brake reservoir pressure too low - 406...
7           Brake blocks to be replaced. X4 (See long).
8            15/04 - AWS Unsolicited brake demand - 401
9           407 - wheel 4 parking brake cylinder damage
10               404 Brake resister trunking broken off
12                         Re-Instate Brake Smart Valve
13                         Re-Instate Brake Smart Valve
14    21/03  -  Blank HMI, DMI and Emergency Brake a...
15    Coach 410 Displaying emergency brake applied o...
16            Brake components  - Brake block - Replace
17                         Re-Instate Brake Smart Valve
20                         carry out brake test RRBX006
21                         Re-Instate Brake Sm

,wonum,description,ldtext,mats_assigned,wopriority,actstart,clean_ldtext,clean_description,clean_ldtext_size,clean_description_size,clean_description_clean_ldtext,all_relevant_mats,count,group_id,similar_clean_description_ldtext,exact_match,wonum_fuzz_matches
54,1427950,Multiple parking brake hoses rubbing,locations are as follows: 411 bogie 1 411 bogi...,NaN,9.0,2019-01-28 03:38:25,"['location', 'follows', 'bogie', 'bogie', 'bog...","['multiple', 'parking', 'brake', 'hose', 'rubb...",7,5,"['multiple', 'parking', 'brake', 'hose', 'rubb...",[],NaN,NaN,NaN,0,"1427950,TLW2261346,TLW1555509,TLW2071536,12926..."
1982,TLW1535017,Multiple parking brake hoses rubbing,Parking brake hoses rubbing at the following l...,NaN,3.0,2019-07-18 17:10:00,"['parking', 'brake', 'hose', 'rubbing', 'follo...","['multiple', 'parking', 'brake', 'hose', 'rubb...",6,5,"['multiple', 'parking', 'brake', 'hose', 'rubb...",[],NaN,NaN,NaN,0,"1427950,TLW2261346,TLW1555509,TLW2071536,12926..."
400,TLW2071536,Multiple parking brake hoses rubbing (see long),402 axle 4 - rubbed through to re-enforcement ...,NaN,9.0,2022-07-22 19:10:25,"['axle', 'rubbed', 'reenforcement', 'axle']","['multiple', 'parking', 'brake', 'hose', 'rubb...",4,7,"['multiple', 'parking', 'brake', 'hose', 'rubb...",[],NaN,NaN,NaN,0,"1427950,1408635,TLW1572826,TLW2071536,TLW20533..."


In [ ]:
df['tf_idf_wonum_match'] = None
df['tf_idf_wonum_match'] = df.apply(lambda row: get_similar_wonums(q_sim_df,df, index_key=row.name, similarity=0.75), axis=1)

In [ ]:
def doc2vec(test_df: pd.DataFrame, text_col: str, vs: int, e: int, win: int, sim_thr: float):

    '''
    Input:  
    text_col: The column of the df which contains the tokenised clean text.
              This column should have the form '['side', 'b', 'bogie',...]' or '[]'
    vs: Vector size parameter of Doc2Vec
    e: Epochs parameter of Doc2Vec
    win: Window parameter of Doc2Vec

    Output:
    model: A Doc2Vec model trained on corpus provided with params provided and min_count=2
    
    '''
    corpus = test_df[text_col]
    tagged_data = [TaggedDocument(words=ast.literal_eval(doc), tags=[str(i)]) for i, doc in enumerate(corpus)]
    print(f'First couple of Tagged data: {tagged_data[0:2]}')
    print(f'Length of tagged data is {len(tagged_data)}')

    # Initialize Doc2Vec model
    model = Doc2Vec(vector_size=vs, min_count=2, epochs=e, window=win)

    # Build vocabulary
    model.build_vocab(tagged_data)

    # Train the model
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    print(f'Model with vec_size={model.vector_size}, window={model.window} and epochs={model.epochs} has been succefully trained based on {text_col}')
    # Get the document embedding vectors
    #document_vectors = [model.infer_vector(ast.literal_eval(doc)) for doc in corpus]


    test_df[f"similar_{text_col}_wonums"] = ""
    test_df[f"similar_{text_col}_index"] = ""

    # Initialize an empty list to store wonum values
    wonum_list = []
    similar_wonums = []
    index_list = []
    similar_index = []


    # Loop through each document in test_df
    for doc_id in range(len(test_df)):
        inferred_vector = model.infer_vector(tagged_data[doc_id].words)
        sims = model.dv.most_similar([inferred_vector], topn= int(len(tagged_data)/100))

        # Use boolean indexing to filter similar wonum values
        similar_wonums = [test_df.iloc[int(sim[0])]["wonum"] for sim in sims if (sim[1] > sim_thr)]  #and (int(sims[0][0]) !=  doc_id)
        similar_index = [ int(sim[0]) for sim in sims if (sim[1] > sim_thr)]  #and (int(sims[0][0]) !=  doc_id)
           
        # Append the list of similar wonum values to wonum_list
        wonum_list.append(similar_wonums if len(similar_wonums) != 1 else '')
        index_list.append(similar_index if len(similar_index) != 1 else '')

    # Assign the array to the new column in test_df
    test_df[f"doc2vec_similar_{text_col}_wonums"] = np.array([",".join(map(str, row)) for row in wonum_list ])
    test_df[f"doc2vec_similar_{text_col}_index"] = np.array([",".join(map(str, row)) for row in index_list ])
    
    return test_df

In [ ]:
test_df = doc2vec(df, text_col='clean_description', vs=125, win=4, e=50, sim_thr=0.8)
test_df[["description", "ldtext", "doc2vec_similar_clean_description_wonums", "doc2vec_similar_clean_description_index"]].head(3)

In [ ]:
'''
Test for nearest matches:

for doc_id in range(0, 6):

    inferred_vector = model.infer_vector(tagged_data[doc_id].words)
    sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))

    print('\nDocument ({}): «{}»'.format(
        doc_id, ' '.join(tagged_data[doc_id].words)))
    for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('THIRD-MOST', 3), ('FOURTH-MOST', 4), ('FIFTH-MOST', 5), ('LEAST', len(sims) - 1)]:
        # if (sims[index][1] > 0.85):
        print(u'%s %s: «%s»' % (label, sims[index], ' '.join(
            tagged_data[int(sims[index][0])].words)))
'''